<a href="https://www.kaggle.com/code/tommyadams/creating-s-p-500-sectors-with-pytorch-kmeans?scriptVersionId=141757172" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Creating New S&P 500 Sectors with Kmeans


<center><img src='https://preview.redd.it/q51lnf0wru471.png?width=960&crop=smart&auto=webp&s=9c63accb214de5c01e7a685088484798856516e1' height=800px width=1000px></center>

## Project Summary

The stock market is a complex system with many different types of companies. One way to make sense of the stock market is to group stocks into sectors. Sectors are groups of stocks that are similar in terms of their industry or economic activity. This can help investors to view the stock market at a macro economic level, determine which stocks are similar to others, and anticipate price moves for a given stock based on changes in its sector.

The finance industry has traditionally divided the S&P 500 stocks into 11 sectors:

- Energy: Oil and gas companies
- Materials: Mining and metals companies
- Industrials: Manufacturing companies
- Consumer Discretionary: Consumer goods companies that sell non-essential products
- Consumer Staples: Consumer goods companies that sell essential products
- Healthcare: Healthcare companies
- Financials: Banks, insurance companies, and other financial services companies
- Information Technology: Technology companies
- Communication Services: Telecom companies and media companies
- Real Estate: Real estate investment trusts and other real estate companies
- Utilities: Public utility companies

One of the main purposes for the sector groupings is to help an investor limit the price volatility of their portfolio by investing in each of the sectors. To serve this purpose, it is important that stocks within a given sector are more correlated with each other than stocks in other sectors. For example, an increase in energy demand should boost the prices of stocks in the energy sector by a similar amount, but should not necessarily affect the prices of stocks outside the energy sector in the same way. 

In this project, I will use a K-means clustering algorithm to group S&P 500 stocks into clusters based on their 5-day percentage price changes, utilizing PyTorch to implement the K-means algorithm. The goal of this project is to determine if I can come up with better sectors than the existing ones for showing which stocks are correlated with each other.

## Background

K-means clustering is a simple but effective clustering algorithm. It works by first randomly selecting k points in the data set. These points are called the centroids. The algorithm then iteratively assigns each data point to the cluster with the closest centroid. The centroids are then updated to the mean of the points in each cluster. This process is repeated until the centroids no longer move.

PyTorch is a popular deep learning framework that is used for natural language processing, computer vision, and other machine learning tasks. It is also a good choice for clustering tasks because it is fast and efficient.

In [ ]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
device_choice = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Data source is https://www.kaggle.com/datasets/yash16jr/s-and-p-500-all-assets
# The data lists daily trading information for the stocks in the S&P 500 from 2010 to 2023

data = pd.read_csv('/kaggle/input/s-and-p-500-all-assets/SnP500 All assets.csv', dtype=str)

In [ ]:
# Exploring the data set
print(data.describe)

In [ ]:
# Identifying the column types for each stock ticker
matches = data.columns.str.findall(r'(.*)\.')
unique_matches = pd.Series([item for row in matches for item in row]).unique()
print(unique_matches)

In [ ]:
# Condensing the data frame to only have daily stock price closing information
close_columns = [item for row in data.columns.str.findall(r'Close.*') for item in row]
closing_prices = data[close_columns]
closing_prices.columns = closing_prices.iloc[0,:]
closing_prices = closing_prices.drop(index=[0,1]).reset_index(drop=True)
closing_prices = closing_prices.astype(float)
closing_prices.head()

In [ ]:
# Calculating the rolling 5 day ratio of closing prices. Several stocks did not having pricing data going back to 2010 since they were not part of the S&P 500 then. 
# I decided to drop these stocks and their NA values from this analysis, leaving me with 438 stocks.
# If higher portion of stocks had NA values, I would have raised the start year from 2010 to remove NA values for these stocks. 
days_for_price_change = 5
price_change = closing_prices.copy()
price_change = price_change.dropna(axis=1)
price_change1 = price_change.drop(index=list(range(days_for_price_change))).reset_index(drop=True)
price_change2 = price_change.drop(index=list(range(len(price_change)-days_for_price_change,len(price_change)))).reset_index(drop=True)
price_change = price_change1 / price_change2
price_change[:5]

In [ ]:
# Converting data frame to a tensor
data_array = np.array(price_change.T, dtype=np.float16)
data_tensor = torch.from_numpy(data_array)

In [ ]:
# Creating data frame with mapping of stock tickers to current S&P 500 sectors
sectors = pd.DataFrame({'Ticker':price_change.columns})
ticker_data = pd.read_csv('/kaggle/input/sp500sectors/constituents.csv', dtype=str)
summary = sectors.merge(ticker_data, left_on=['Ticker'], right_on=['Symbol'], how='left')
summary = summary.drop(columns=['Symbol'], axis=1)

In [ ]:
# Defining the loss function (aka 'inertia') to measure each grouping of stocks. The lower the inertia, the better the grouping is.
def inertia(data, cluster_centers, cluster_matches):
    centers = torch.stack([cluster_centers[i] for i in cluster_matches])
    diff = (data - centers)**2
    diff_total = diff.sum()**0.5
    return diff_total

In [ ]:
# Calculating the inertia of the existing S&P 500 sectors.
price_change_sp_sectors = price_change.T
price_change_sp_sectors['Sector']=pd.factorize(summary['Sector'])[0]
centers = price_change_sp_sectors.groupby(['Sector']).mean()
sp_sectors_inertia = inertia(data=data_tensor, cluster_centers=torch.tensor(centers.values), cluster_matches=torch.tensor(price_change_sp_sectors['Sector'].values))
print(sp_sectors_inertia)

In [ ]:
# Creating Kmeans algorithm 

def initialize(X, num_clusters, seed, start_clusters):


    if seed == None:
        indices = np.random.choice(len(X), num_clusters, replace=False)
    else:
        np.random.seed(seed)
        indices = np.random.choice(len(X), num_clusters, replace=False)
        
    initial_state = X[indices]
    
    if len(start_clusters) > 0:
        for i in range(len(start_clusters)):
            initial_state[i] = torch.Tensor(start_clusters[i,:])
    
    return initial_state

def kmeans(
        X,
        num_clusters,
        tol=.0001,
        iter_limit=99,
        device=torch.device(device_choice),
        seed=None,
        start_clusters=[]
):
    
    X = X.float()
    X = X.to(device)
    initial_state = initialize(X, num_clusters, seed=seed, start_clusters=start_clusters)
    iteration = 0
   
    while True:

        dis = distance(X, initial_state, device=device)
        cluster_assign = torch.argmin(dis, dim=1)
        initial_state_pre = initial_state.clone()

        for index in range(num_clusters):
            selected = torch.nonzero(cluster_assign == index).squeeze().to(device)
            selected = torch.index_select(X, 0, selected)
            
            # Update cluster centers
            if selected.shape[0] != 0:
                initial_state[index] = selected.mean(dim=0)

        cluster_shift = torch.sum(
            torch.sum((initial_state - initial_state_pre) ** 2, dim=1) ** 0.5
            )

        # increment iteration
        iteration = iteration + 1

        if cluster_shift < tol:
            break
        if iter_limit != 0 and iteration >= iter_limit:
            print('iter_limit reached')
            break

    return cluster_assign.cpu(), initial_state.cpu()

def distance(dataset, clusters, device=torch.device(device_choice)):
    
    # transfer to device
    dataset, clusters = dataset.to(device), clusters.to(device)

    # Stocks x 1 x Dates
    A = dataset.unsqueeze(dim=1)

    # 1 x Clusters x Dates
    B = clusters.unsqueeze(dim=0)

    
    dis = (A - B) ** 2.0
    
    # return Stocks * Clusters matrix for euclidean distance
    dis = dis.sum(dim=-1) ** 0.5
    dis = dis.squeeze()
    return dis


In [ ]:
# Calculating inertia of groupings with different numbers of clusters. 
# The number of clusters is one of the parameters to set with the Kmeans algorithm. Naturally, inertia decreases the more cluster there are,
# but there may be a critical point at which the marginal benefits of adding new clusters are diminished. Based on the graph, 
# it does not look there is a critical point.

clusters_list = []
cluster_matches_list = []
inertias = []

for cluster_count in range(2,30):
    cluster_ids_x, cluster_centers = kmeans(
    X=data_tensor, num_clusters=cluster_count, tol=.0001, seed=np.random.seed(100)
    )
    clusters_list.append(cluster_centers)
    cluster_matches_list.append(cluster_ids_x)
    
    inertia_current = inertia(data=data_tensor, cluster_centers=cluster_centers, cluster_matches=cluster_ids_x)
    inertias.append(inertia_current)
    
plt.plot(range(2,30), inertias)
plt.show()

In [ ]:
# Creating a function to run the Kmeans algorithm multiple times. In most situations, including this one, the Kmeans algorithm has multiple relative minimums.
# To find groupings with low inertia, I used this function to run Kmeans multiple times and then select the grouping with the lowest inertia.
# For better performance, set the torch device to gpu when calling kmeans_iterate. 

def kmeans_iterate(data, num_clusters, iterations, start_clusters=[], tol=.0001, device=torch.device(device_choice), seed=None): 

    clusters_list = []
    cluster_matches_list = []
    inertias = []

    for i in range(iterations):
        if seed != None:
            seed = seed+1
        cluster_ids_x, cluster_centers = kmeans(
        X=data, num_clusters=num_clusters, tol=tol, start_clusters=start_clusters, device=device, seed=seed
        )
        clusters_list.append(cluster_centers)
        cluster_matches_list.append(cluster_ids_x)

        inertia_current = inertia(data=data_tensor, cluster_centers=cluster_centers, cluster_matches=cluster_ids_x)
        inertias.append(inertia_current)

    
    
    return clusters_list, cluster_matches_list, inertias

In [ ]:
# I ran the Kmeans formula 50 times with 11 clusters and graphed the grouping with the lowest inertia.
# This inertia is lower than the 37.7590 inertia of the current sectors
clusters_list, cluster_matches_list, inertias = kmeans_iterate(data_tensor, num_clusters=11, iterations=50, seed=534) 
lowest_inertia_index = inertias.index(min(inertias))
summary['Kmeans_Sector'] = cluster_matches_list[lowest_inertia_index] + 1
summary.groupby(['Kmeans_Sector', 'Sector'])['Kmeans_Sector'].count().unstack('Sector').plot(kind='bar', stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.show()
print(min(inertias))

In [ ]:
# Normally, the starting points for the clusters are randomly selected from the existing data points. In the code below, I tried switching the starting points so that they
# were closer to the means of the data points, but this did not significantly improve the inertia of the results, so I elected to not set the 
# cluster start points in later iterations. 

data_mean = data_array.mean(axis=0)
num_start_clusters = 11
cluster_locs = np.array([data_mean + np.random.randint(-100,100) * .00001 for i in range(num_start_clusters)])
clusters_list, cluster_matches_list, inertias = kmeans_iterate(data_tensor, num_clusters=num_start_clusters, iterations=50, start_clusters=cluster_locs) 
lowest_inertia_index = inertias.index(min(inertias))
summary['Kmeans_Sector'] = cluster_matches_list[lowest_inertia_index] + 1
summary.groupby(['Kmeans_Sector', 'Sector'])['Kmeans_Sector'].count().unstack('Sector').plot(kind='bar', stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.show()
print(min(inertias))

In [ ]:
# Running the Kmeans algorithm 500 times with 7 clusters. I decided to use 7 clusters since it looked like there were multiple small clusters
# when 11 clusters were used previously. Lowering the cluster count also helps reveal which industries are more similar to each other.
clusters_list, cluster_matches_list, inertias = kmeans_iterate(data_tensor, num_clusters=7, iterations=500, seed=1000) 

In [ ]:
# Graphing the 9 groupings with the lowest inertias
nrows = 3
ncols = 3
partition_array = np.argsort(inertias)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(11,11))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
for row in range(nrows):
    for col in range(ncols):
        inertia_index = partition_array[3*row+col]
        summary['Kmeans_Sector'] = cluster_matches_list[inertia_index] + 1
        summary.groupby(['Kmeans_Sector', 'Sector'])['Kmeans_Sector'].count().unstack('Sector').plot(kind='bar', stacked=True, ax=axes[row, col])
        axes[row,col].set_xlabel('')
        axes[row,col].set_title(f'Inertia: {round(float(inertias[inertia_index]),3)}')
        if row == 0 and col == ncols - 1:
            axes[row,col].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', reverse=True)
        else:
            axes[row,col].get_legend().remove()
plt.show()

## Graphs
I showed several stacked bar charts of the groupings that resulted from the Kmeans algorithm. In each bar chart, the y-axis is the number of stocks in each cluster and the x-axis is the cluster number for each cluster. The legend color labels correspond to how the finance industry has currently classified each stock into a sector. 


## Results

In the first bar chart, was able to outperform the current sector groupings by generating a lower inertia with 11 clusters than the inertia of the current sector groupings. This means that my clustering algorithm was able to group the stocks more effectively than the traditional sector groupings.

After examining the results, I noticed that there were multiple clusters that were quite small. This implied that a smaller number of clusters might be a better fit.

I then ran more iterations with 7 clusters and found a result that still had less inertia than the current sectors despite having 4 less groups. This is a significant improvement, and suggests that the clustering algorithm is able to identify meaningful patterns in the data that the traditional sector groupings do not.

I also looked at the 9 iteration groupings with the lowest inertias. From the bar charts of their groupings, I made the following observations:

- The energy sector is consistently its own group, showing that energy stocks tend to be some of the least correlated stocks to the rest of the market and that this current sector grouping is justified.

- Utilities and real estate industries tend to be grouped together; this makes sense because both sectors have large assets that produce consistent dividends.

- In a majority of the 9 iterations, consumer staples is also in the same group as utilities and real estate. This makes sense because all three of these sectors are somewhat conservative investments in that they produce reliable cash flows but do not have much potential for large gains.

- The industrial and material sectors are often grouped together, probably because the demand for industrial goods also affects the demand for the raw materials that the industrial companies use.

- In half of the 9 iterations, there is a large chunk of financials that is also grouped with the industrial and material sectors. My guess is that this occurs because these are all somewhat cyclical sectors that are tied to the business cycle; when the economy is expanding, there is an increasing demand for goods which leads to expansions in industrials, materials, and financing.

- The rest of the sectors (information technology, health care, consumer discretionary, communication services) tend to be more dispersed across the groupings, implying that these sectors are more diverse and that the stocks in these sectors are not as strongly correlated with each other.

- In particular, consumer discretionary and communication services are consistently dispersed across several groups, implying that these sectors should be removed or reworked to identify a group of stocks that are more correlated with each other.

In conclusion, I was able to generate stock groupings that better reflected price movement than the existing sectors defined by the finance industry. My recommendation for improving the existing sectors is to rework the consumer discretionary and communication services sectors and possibly the health care and information technology sectors.